In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

In [23]:
url_parafina = "https://parafina.eco/tienda/gafas-de-sol-ecologicas/"

res = requests.get(url_parafina)
print(res.status_code)
sopa = BeautifulSoup(res.content, "html.parser")

elementos_gafas = sopa.find("ul", {"class":"products columns-3"})
#print(elementos_gafas)

# Encuentra todos los productos en la lista
productos = elementos_gafas.find_all('li', class_='product')
print(len(productos))

dic_parafina = {
    "nombre":[],
    "precio":[],
    "colores":[],
    "material":[],
}

# Iterar sobre cada producto y extraer la información
for producto in productos:

    nombre_producto = producto.find('h4').get_text(strip=True) if producto.find('h4') else "Nombre no disponible"

    # Extraer el precio del producto
    precio = producto.find('span', class_='price').get_text(strip=True) if producto.find('span', class_='price') else "Precio no disponible"
    precio_final = precio.split()[-1]

    # Extraer los colores disponibles
    colores = [li['data-tooltip'] for li in producto.select('.frame_color_list .frame_color_variation')]

    # Categoria del producto
    categoria = producto.find("small", {"class":"product_category"}).text.strip()

    dic_parafina["nombre"].append(nombre_producto)
    dic_parafina["precio"].append(precio_final)
    dic_parafina["colores"].append(colores)
    dic_parafina["material"].append(categoria)

    # Imprimir resultados para este producto
    print("Nombre del producto:", nombre_producto)
    print("Precio:", precio_final)
    print("Colores disponibles:", colores)
    print("Material producto: ", categoria)
    print("---")  # Separador entre productos

    

200
9
Nombre del producto: Isla
Precio: 31,85€.
Colores disponibles: ['Atardecer', 'Lluvia', 'Carey Genjibre', 'Musgo Mate', 'Negro Mate', 'Rosa Palo', 'Carey']
Material producto:  PET (Plástico) Reciclado
---
Nombre del producto: Cala
Precio: 38,35€.
Colores disponibles: ['Transparente', 'Atardecer', 'Caramelo', 'Negro', 'Carey Suave']
Material producto:  Bambú Orgánico
---
Nombre del producto: Arroyo
Precio: 31,85€.
Colores disponibles: ['Lluvia', 'Carey Genjibre', 'Transparente', 'Negro Mate', 'Rosa Palo', 'Carey']
Material producto:  PET (Plástico) Reciclado
---
Nombre del producto: Costa
Precio: 38,35€.
Colores disponibles: ['Transparente', 'Zanahoria', 'Caramelo', 'Negro', 'Carey Suave']
Material producto:  Bambú Orgánico
---
Nombre del producto: Cauce
Precio: 31,85€.
Colores disponibles: ['Atardecer', 'Lluvia', 'Carey Genjibre', 'Transparente', 'Negro Mate', 'Rosa Palo']
Material producto:  PET (Plástico) Reciclado
---
Nombre del producto: Ruta
Precio: 38,35€.
Colores disponible

In [21]:
pd.DataFrame(dic_parafina)

,nombre,precio,colores,material
0,Isla,"31,85€.","[Atardecer, Lluvia, Carey Genjibre, Musgo Mate...",PET (Plástico) Reciclado
1,Cala,"38,35€.","[Transparente, Atardecer, Caramelo, Negro, Car...",Bambú Orgánico
2,Arroyo,"31,85€.","[Lluvia, Carey Genjibre, Transparente, Negro M...",PET (Plástico) Reciclado
3,Costa,"38,35€.","[Transparente, Zanahoria, Caramelo, Negro, Car...",Bambú Orgánico
4,Cauce,"31,85€.","[Atardecer, Lluvia, Carey Genjibre, Transparen...",PET (Plástico) Reciclado
5,Ruta,"38,35€.","[Transparente, Negro, Verde Oscuro]",Goma Reciclada
6,Camino,"38,35€.","[Transparente, Negro, Rosa Palo]",Goma Reciclada
7,Colina,"44,85€.","[Havana, Crema, Rubor Brillante, Negro, Carey]",HDPE (Plástico) Reciclado
8,Valle,"31,85€.","[Atardecer, Lluvia, Negro Mate]",PET (Plástico) Reciclado
